In [276]:
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

In [237]:
def load(chunk):
    # load dataframe from feature engineering
    print('Reading JSON chunk', chunk)
    fp = 'urban_sound_classification_feature_eng_' + str(chunk) + '.json'
    df = pd.read_json(fp) 
    return df

In [238]:
df = [load(i) for i in range(11)]
df[0].head()

Reading JSON chunk 0
Reading JSON chunk 1
Reading JSON chunk 2
Reading JSON chunk 3
Reading JSON chunk 4
Reading JSON chunk 5
Reading JSON chunk 6
Reading JSON chunk 7
Reading JSON chunk 8
Reading JSON chunk 9
Reading JSON chunk 10


,Audio_Windowed,Energy_Windowed,Label
0,"[-0.0057176819, -0.0011095886, 0.003245285, -0...","[226558.6064070214, 107337.3012845638, 135170....",5
1,"[-0.0030328674, -0.0010735931, -0.005166946400...","[387147.0688923378, 425738.9471584653, 249002....",6
10,"[0.0126753206, 0.0062408283, 0.006772960200000...","[878281.4858788522, 189916.9570646946, 132660....",1
100,"[-0.0001343026, 4.39227e-05, 6.054e-06, 8.5151...","[5471.5342768834, 4120.1259257783, 8891.585204...",5
101,"[0.0732835236, 0.073182724, 0.0723233795, 0.07...","[12428.0586252659, 1694.5723434308, 9024.89557...",4


In [239]:
features = pd.concat([df[0], df[1]])
for i in range(2,11):
    features = pd.concat([features, df[i]])
len(features)

5435

In [360]:
data = [f[1] + f[2] + [f[3]] for f in features.itertuples()]
random.seed(123)
random.shuffle(data)
train_size = int(.8 * len(data))
train_X, train_Y = np.array([i[:-1] for i in data[:train_size]]), to_categorical([i[-1] for i in data[:train_size]])
test_X, test_Y = np.array([i[:-1] for i in data[train_size:]]), to_categorical([i[-1] for i in data[train_size:]])

In [361]:
# create MLP model
nodes, rate = 192, 0.1
model = Sequential()
model.add(Dense(nodes, activation='tanh', input_dim=len(train_X[0])))
model.add(Dropout(rate))
model.add(Dense(nodes, activation='relu'))
model.add(Dropout(rate))
model.add(Dense(nodes, activation='tanh'))
model.add(Dropout(rate))
model.add(Dense(nodes, activation='relu'))
model.add(Dropout(rate))
model.add(Dense(10, activation='softmax'))

In [362]:
# compile and fit model to show no overfitting
batch, eps = 64, 50
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(train_X, train_Y, epochs=eps, batch_size=batch)
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))
model_Y = model.predict(test_X)
confusion_matrix(test_Y.argmax(axis=1), model_Y.argmax(axis=1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_290 (Dense)            (None, 192)               18624     
_________________________________________________________________
dropout_109 (Dropout)        (None, 192)               0         
_________________________________________________________________
dense_291 (Dense)            (None, 192)               37056     
_________________________________________________________________
dropout_110 (Dropout)        (None, 192)               0         
_________________________________________________________________
dense_292 (Dense)            (None, 192)               37056     
_________________________________________________________________
dropout_111 (Dropout)        (None, 192)               0         
_________________________________________________________________
dense_293 (Dense)            (None, 192)               37056     
__________

array([[ 11,   8,   1,   1,   0,   0,   9,  11,   4,   0],
       [  0,  88,   2,   0,   1,   1,   4,  10,   5,   0],
       [  6,  19,  42,   6,   5,   3,  13,   9,  11,   1],
       [  2,   6,   2,  76,   8,   7,   6,   8,   9,   3],
       [  3,  13,   3,   5,  66,   5,  12,   9,   0,   1],
       [  1,   4,   5,   1,   1,  83,   6,   0,   1,   2],
       [  6,  12,  10,   4,   9,   1,  67,  13,   8,   0],
       [ 11,  13,   1,   0,   0,   1,   3, 103,   0,   1],
       [  1,   7,   1,   2,   0,   1,  10,   6, 106,   2],
       [  2,  12,   3,   0,  13,   1,   8,   2,   3,  25]])

In [353]:
# create and train Random Forest model
clf = RandomForestClassifier(n_estimators=100)
clf.fit(train_X, train_Y)
print ("Test score:", clf.score(test_X, test_Y))
clf_Y = clf.predict(test_X)
confusion_matrix(test_Y.argmax(axis=1), clf_Y.argmax(axis=1))

Test score: 0.48206071757129715


array([[ 44,   0,   0,   0,   0,   0,   0,   0,   0,   1],
       [ 35,  76,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 81,   0,  32,   1,   0,   1,   0,   0,   0,   0],
       [ 50,   0,   0,  75,   1,   0,   0,   0,   1,   0],
       [ 76,   1,   0,   0,  39,   0,   0,   0,   0,   1],
       [ 41,   0,   0,   0,   0,  63,   0,   0,   0,   0],
       [110,   0,   1,   0,   0,   0,  19,   0,   0,   0],
       [ 26,   0,   0,   0,   0,   0,   0, 103,   4,   0],
       [ 53,   0,   0,   1,   0,   0,   0,   0,  82,   0],
       [ 44,   0,   0,   1,   6,   0,   0,   0,   0,  18]])

In [347]:
# normalize
scaler = MinMaxScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.fit_transform(test_X)

In [363]:
# rerun with normalization
print('NN Results:')
model.fit(train_X_scaled, train_Y, epochs=eps, batch_size=batch)
loss, acc = model.evaluate(test_X_scaled, test_Y, batch_size=batch)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

NN Results:
Epoch 1/50
4348/4348 [==============================] - 0s 95us/step - loss: 2.2649 - acc: 0.1858
Epoch 2/50
4348/4348 [==============================] - 0s 87us/step - loss: 2.0467 - acc: 0.2470
Epoch 3/50
4348/4348 [==============================] - 0s 88us/step - loss: 1.9555 - acc: 0.2900
Epoch 4/50
4348/4348 [==============================] - 0s 96us/step - loss: 1.8564 - acc: 0.3192
Epoch 5/50
4348/4348 [==============================] - 0s 84us/step - loss: 1.7837 - acc: 0.3661
Epoch 6/50
4348/4348 [==============================] - 0s 83us/step - loss: 1.7140 - acc: 0.3875
Epoch 7/50
4348/4348 [==============================] - 0s 85us/step - loss: 1.6555 - acc: 0.4103
Epoch 8/50
4348/4348 [==============================] - 0s 114us/step - loss: 1.6412 - acc: 0.4197
Epoch 9/50
4348/4348 [==============================] - 0s 109us/step - loss: 1.6048 - acc: 0.4434
Epoch 10/50
4348/4348 [==============================] - 0s 84us/step - loss: 1.5496 - acc: 0.4508
Epoch

In [349]:
# rerun with normalization
print('RF Results:')
clf.fit(train_X_scaled, train_Y)
print ("Test score:", clf.score(test_X_scaled, test_Y))

RF Results:
Test score: 0.08095676172953081
